In [103]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report
import numpy as np
import cv2
import os
import tqdm

In [35]:
PATH_COVID = './dataset/train/covid/'
PATH_NORMAL = './dataset/train/normal/'

In [36]:
covid_list = os.listdir(PATH_COVID)
normal_list = os.listdir(PATH_NORMAL)

In [37]:
def load_reshape_img(image):
    img = load_img(image, target_size=(224, 224))
    x = img_to_array(img)/255.
    x = x.reshape((1,) + x.shape)
    return x


## Image preproccesing:

In [39]:
datagen = ImageDataGenerator(
                                    rescale=1./255,
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    brightness_range=[0.2,1.0],
                                    shear_range=0.2,
                                    zoom_range=[0.5,1.0],
                                    fill_mode='nearest')

for xr in tqdm.tqdm(covid_list):
    X = np.array([load_reshape_img(PATH_COVID + image) for image in covid_list])
    X = X.reshape(X.shape[0], 224, 224, 3)    
    i = 0
    for batch in datagen.flow(X, batch_size=70,
                              save_to_dir=PATH_COVID, 
                              save_prefix='covid', 
                              save_format='jpeg'):        
        i += 1
        if i > 10:
            break


100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [08:24<00:00, 10.74s/it]


In [40]:
for xr in tqdm.tqdm(normal_list):
    X = np.array([load_reshape_img(PATH_NORMAL + image) for image in normal_list])
    X = X.reshape(X.shape[0], 224, 224, 3)    
    i = 0
    for batch in datagen.flow(X, batch_size=70,
                              save_to_dir=PATH_NORMAL, 
                              save_prefix='normal', 
                              save_format='jpeg'):        
        i += 1
        if i > 10:
            break

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [02:07<00:00,  6.35s/it]


## Model training:

In [63]:
model_v3 = InceptionV3(weights='imagenet', include_top=False, classes=2, input_shape=(224, 224, 3))
new_layers = model_v3.output

new_layers = GlobalAveragePooling2D()(new_layers)

new_layers = Dense(128, activation='relu')(new_layers)
new_layers = Dropout(0.5)(new_layers)
new_layers = BatchNormalization()(new_layers)

new_layers = Dense(2, activation='softmax')(new_layers)
model_v3 = Model(inputs=model_v3.inputs, outputs=new_layers)

In [64]:
# Freezing the first 51 layers
for layer in model_v3.layers[:52]:
    layer.trainable = False

In [65]:
model_v3.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='sparse_categorical_crossentropy', metrics=['accuracy']) #Stochastic gradient descent optimizer.


In [66]:
train_datagen = ImageDataGenerator(validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
        './dataset/train/',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        './dataset/train/', # same directory as training data
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',
        subset='validation') # set as validation data

model_v3.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // 32,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // 32,
    epochs = 2)

Found 22485 images belonging to 2 classes.
Found 5621 images belonging to 2 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 702 steps, validate for 175 steps
Epoch 1/2
702/702 [==============================] - 3581s 5s/step - loss: 0.3781 - accuracy: 0.8383 - val_loss: 76.9682 - val_accuracy: 0.8450
Epoch 2/2
702/702 [==============================] - 3468s 5s/step - loss: 0.0976 - accuracy: 0.9829 - val_loss: 76.3031 - val_accuracy: 0.8450


### Saving the model:

In [67]:
def save_model(model):
    # serialize model to JSON
    with open(f"{model}.json", "w") as json_file:
        json_file.write(model_v3.to_json())

    # serialize weights to HDF5
    model_v3.save_weights(f"{model}.h5")
    print("Saved model to disk")
save_model('model_2')    

Saved model to disk


## Evaluate on the test data:

In [107]:
PATH_TEST = './dataset/valid/'
test_list = os.listdir(PATH_TEST)

Xtest = np.array([load_reshape_img(PATH_TEST + image) for image in test_list])
Xtest = Xtest.reshape(Xtest.shape[0], 224, 224, 3)
ytest = []
for i in test_list:
    if 'normal' in i:
        ytest.append(1)
    else:
        ytest.append(0)

In [108]:
y_pred = model_v3.predict(Xtest, batch_size=64, verbose=0)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(ytest, y_pred_bool, target_names=['covid', 'normal']))

              precision    recall  f1-score   support

       covid       0.96      1.00      0.98        23
      normal       1.00      0.88      0.93         8

    accuracy                           0.97        31
   macro avg       0.98      0.94      0.96        31
weighted avg       0.97      0.97      0.97        31

